In [ ]:
%%sh
# Copy dgl sample folder and move to inside the folder
cp amazon-sagemaker-examples/sagemaker-python-sdk/dgl_gcn_tox21/gcn_tox21_gpu.Dockerfile ./
cp amazon-sagemaker-examples/sagemaker-python-sdk/dgl_gcn_tox21/main.py ./

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session

# Setup session
sess = sagemaker.Session()

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = 'dgl-tag-analysis'

# Location to put your custom code.
custom_code_upload_location = 'sagemaker/dgl-tox21/customcode'

# IAM execution role that gives Amazon SageMaker access to resources in your AWS account.
# You can use the Amazon SageMaker Python SDK to get the role from the notebook environment. 
role = get_execution_role()

In [ ]:
# Show script main.py
# !cat main.py

In [ ]:
%%sh


In [ ]:
%%sh
# # For CPU default_docker_name="dgllib/dgl-sagemaker-cpu:dgl_0.4_pytorch_1.2.0_rdkit"
default_docker_name="dgllib/dgl-sagemaker-gpu:dgl_0.4_pytorch_1.2.0_rdkit"
# docker pull $default_docker_name

# Build docker image from .Dockerfile
docker_name=sagemaker-dgl-pytorch-gcn-tox21

# For CPU docker build -t $docker_name -f gcn_tox21_cpu.Dockerfile .
# sudo docker build -t $docker_name -f gcn_tox21_gpu.Dockerfile .
account=$(aws sts get-caller-identity --query Account --output text)
echo $account
region=$(aws configure get region)

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${docker_name}:latest"
# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${docker_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${docker_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

docker tag ${docker_name} ${fullname}
docker push ${fullname}

In [ ]:
import boto3

# Set target dgl-docker name
docker_name='sagemaker-dgl-pytorch-gcn-tox21'

CODE_PATH = 'main.py'
code_location = sess.upload_data(CODE_PATH, bucket=bucket, key_prefix=custom_code_upload_location)

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, docker_name)
print(image)

estimator = sagemaker.estimator.Estimator(image,
                        role, 
                        train_instance_count=1, 
                        train_instance_type= 'ml.p3.2xlarge', #'ml.c4.2xlarge'
                        hyperparameters={'entrypoint': CODE_PATH},
                        sagemaker_session=sess)

In [ ]:
estimator.fit({'training-code': code_location})